<div style="background: rgb(232, 232, 255); padding-left: 10px; border: 1px solid black; font-family: Monospace; border-radius: 15px; box-shadow: 2px 2px 0px 0px rgba(0,0,0,0.7);">

# Dataset Creation

I am going to create a database with a huge number of songs, storing their **features**, as well as the **lyrics**.<br><br>
Steps I am going to take in order to create a database:<br>

- Scrape **TOP 100 Songs** for every week across ~60 years from **www.officialcharts.com**
- For every scraped song, search for the song using **Spotify API**
- For every found song, store it's info (album, artist, ...)
- For every distinct **album** save all the tracks
- For every song, extract it's **technical features**

<img src="db_management/other/ERD_songs.JPG" width=1100>


---

In [1]:
import pandas as pd
import numpy as np
import requests
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import os
from datetime import datetime, timedelta
from time import sleep
from bs4 import BeautifulSoup
from typing import Tuple, List, NamedTuple
from db_management.db import SongInfo, IDSongInfo, SongsContainer, SongsDB, DBException, ArtistInfo, AlbumInfo, SongFeatures

<div style="background: rgb(232, 232, 255); padding-left: 10px; border: 1px solid black; font-family: Monospace; border-radius: 15px; box-shadow: 2px 2px 0px 0px rgba(0,0,0,0.7);">

### Connecting with the Spotify API

In [2]:
CLIENT_ID = os.environ["SPOTIFY_CLIENT_ID"]
CLIENT_SECRET = os.environ["SPOTIFY_CLIENT_SECRET"]

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID, client_secret=CLIENT_SECRET, redirect_uri='http://example.com', scope="playlist-modify-public"))

<div style="background: rgb(232, 232, 255); padding-left: 10px; border: 1px solid black; font-family: Monospace; border-radius: 15px; box-shadow: 2px 2px 0px 0px rgba(0,0,0,0.7);">

### Function generating the date range in given format

In [3]:
ENDPOINT_CHARTS = "https://www.officialcharts.com/charts/singles-chart/%s/7501/"

def generate_dates(week_gap: int = 2, years_back: int = 50):

    today = datetime.now()
    start_date = today - timedelta(days=365*years_back)
    print(len([n for n in range(int((today-start_date).days//(week_gap*7)))]))
    
    for n in range(int((today - start_date).days//(week_gap*7))):
        yield start_date + timedelta(days=n*week_gap*7)

<div style="background: rgb(232, 232, 255); padding-left: 10px; border: 1px solid black; font-family: Monospace; border-radius: 15px; box-shadow: 2px 2px 0px 0px rgba(0,0,0,0.7);">

## Scraping TOP 100 songs across 60 years

I made this step before creating a database, so here I store the results to the **CSV** file, however after the database was implemented, I just created a function to populate the table, using the created csv.

In [4]:
def retrieve_top_songs(date: str, container: SongsContainer) -> SongsContainer:
    try:
        response = requests.get(ENDPOINT_CHARTS % date)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        items = soup.find_all("div", {"class": "chart-item"})

        for item in items:
            try:
                song = item.find("a", {"class": "chart-name"}).find("span", {"class": None})
                artist = item.find("a", {"class": "chart-artist"}).find("span", {"class": None})
    
                container.add_song(SongInfo(artist.text, song.text))
            except:
                continue
        return container
        
    except Exception as exception:
        print(exception)

In [107]:
songs = SongsContainer()
for idx, single_date in enumerate(generate_dates(week_gap=1, years_back=60)):
    date = single_date.strftime("%Y%m%d")
    retrieve_top_songs(date, songs)
    if idx % 101 == 100:
        print(idx, date)

3128
100 19660615
201 19680522
302 19700429
403 19720405
504 19740313
605 19760218
706 19780125
807 19800102
908 19811209
1009 19831116
1110 19851023
1211 19870930
1312 19890906
1413 19910814
1514 19930721
1615 19950628
1716 19970604
1817 19990512
1918 20010418
2019 20030326
2120 20050302
2221 20070207
2322 20090114
2423 20101222
2524 20121128
2625 20141105
2726 20161012
2827 20180919
2928 20200826
3029 20220803


In [108]:
len(songs)

43220

<div style="background: rgb(232, 232, 255); padding-left: 10px; border: 1px solid black; font-family: Monospace; border-radius: 15px; box-shadow: 2px 2px 0px 0px rgba(0,0,0,0.7);">

### Connecting the database

Here I also populate the table with the scraped songs (title and artist).

In [3]:
database = SongsDB()

In [6]:
database.songs_populate_csv("songs2.csv")

In [4]:
for idx, title, artist in database.get_scraped_songs()[:5]:
    print(title, artist)

HOUSE OF THE RISING SUN THE ANIMALS
IT'S ALL OVER NOW THE ROLLING STONES
HOLD ME P J PROBY
IT'S OVER ROY ORBISON
RAMONA THE BACHELORS


<div style="background: rgb(232, 232, 255); padding-left: 10px; border: 1px solid black; font-family: Monospace; border-radius: 15px; box-shadow: 2px 2px 0px 0px rgba(0,0,0,0.7);">

## Searching for the songs using Spotify API

The only problem was the **rate limit**, which I dodged using the sleep method (180 requests per minute), however Spotify implemented some other limits, not sharing info about, so I got freezed, and therefore had to continue the next day.

In [7]:
def get_song_id(s: SongInfo) -> IDSongInfo:
    ext_info = sp.search(q=f"track: {s.song} artist: {s.artist}", type='track', limit=1)
    item = ext_info["tracks"]["items"][0]
    
    song_id = item["id"]
    album_id = item["album"]["id"]
    artist_id = item["artists"][0]["id"]
    title = item["name"]
    release_date = item["album"]["release_date"]
    featured = int(len(item["artists"]) > 1)
    popularity = item["popularity"]

    return IDSongInfo(song_id, album_id, artist_id, title, release_date, featured, popularity)

In [8]:
start = datetime.now()

skipped = 0

for idx, title, artist in database.get_scraped_songs()[24400:]:
    try:
        database.songs_insert(get_song_id(SongInfo(title, artist)))
        if idx % 50 == 0 and idx != 0:
            print(f"{idx}: cooldown 5s... TIME:{datetime.now()-start} SKIPPED: {skipped}")
            sleep(5)
    except DBException as exception:
        skipped += 1
        continue
    except Exception as gen_exception:
        print(gen_exception)
        sleep(10)

24450: cooldown 5s... TIME:0:00:17.342091 SKIPPED: 39
24500: cooldown 5s... TIME:0:00:40.385582 SKIPPED: 79
24650: cooldown 5s... TIME:0:01:36.293843 SKIPPED: 192
24850: cooldown 5s... TIME:0:02:49.205855 SKIPPED: 349
24950: cooldown 5s... TIME:0:03:27.405053 SKIPPED: 427
25000: cooldown 5s... TIME:0:03:50.034166 SKIPPED: 461
25100: cooldown 5s... TIME:0:04:28.863586 SKIPPED: 492
25150: cooldown 5s... TIME:0:04:52.000217 SKIPPED: 503
25200: cooldown 5s... TIME:0:05:14.106779 SKIPPED: 519
25250: cooldown 5s... TIME:0:05:37.031791 SKIPPED: 532
25300: cooldown 5s... TIME:0:05:59.165291 SKIPPED: 544
25400: cooldown 5s... TIME:0:06:40.137510 SKIPPED: 570
25450: cooldown 5s... TIME:0:07:03.065331 SKIPPED: 583
25500: cooldown 5s... TIME:0:07:28.264610 SKIPPED: 596
25550: cooldown 5s... TIME:0:07:53.641632 SKIPPED: 608
25850: cooldown 5s... TIME:0:09:48.652726 SKIPPED: 679
25950: cooldown 5s... TIME:0:10:31.560313 SKIPPED: 708
26000: cooldown 5s... TIME:0:10:55.938681 SKIPPED: 722
26050: coold

<div style="background: rgb(232, 232, 255); padding-left: 10px; border: 1px solid black; font-family: Monospace; border-radius: 15px; box-shadow: 2px 2px 0px 0px rgba(0,0,0,0.7);">

## Additional step, getting additional songs from playlists

I wanted to test another API feature, getting more vintage songs (got around 2000).

In [10]:
playlists_names = [f"{year} hits" for year in range(1950, 2010, 10)]
playlists_names

['1950 hits', '1960 hits', '1970 hits', '1980 hits', '1990 hits', '2000 hits']

In [8]:
def get_playlists_songs(query: str):
    skipped = 0
    total = 0
    response = sp.search(q=query, type='playlist', limit=10)
    for it in response["playlists"]["items"]:
        name = it["name"].lower()
        if "polsk" not in name and "lat" not in name and "hity" not in name and "piosenk" not in name:
            try:
                id = it["id"]
                playlist = sp.playlist(id)
            except Exception as exception_playlist:
                print(exception_playlist)
                continue
                
            for it in playlist["tracks"]["items"]:
                total += 1
                try:
                    it = it["track"]
                    album_id = it["album"]["id"]
                    artist_id = it["artists"][0]["id"]
                    song_id = it["id"]
                    popularity = it["popularity"]
                    featured = int(len(it["artists"]) > 1)
                    title = it["name"]
                    release_date = it["album"]["release_date"]
    
                    database.songs_insert(IDSongInfo(song_id, album_id, artist_id, title, release_date, featured, popularity))
                except DBException as exception_db:
                    skipped += 1
                    continue
                except Exception as exception_general:
                    print(exception_general)
                    continue
    print(f"Added {total-skipped} out of {total}")

In [55]:
for query in playlists_names:
    get_playlists_songs(query)

Added 56 out of 640
Added 177 out of 725
Added 214 out of 533
Added 69 out of 608
Added 252 out of 700
Added 209 out of 773


In [57]:
for query in ["old english songs", "old rap", "old hiphop", "old pop", "micheal jacson", "elvis presley"]:
    get_playlists_songs(query)

Added 266 out of 641
Added 322 out of 596
Added 295 out of 660
Added 469 out of 1000
Added 79 out of 391
Added 211 out of 545


<div style="background: rgb(232, 232, 255); padding-left: 10px; border: 1px solid black; font-family: Monospace; border-radius: 15px; box-shadow: 2px 2px 0px 0px rgba(0,0,0,0.7);">

## Populating artists table

In [9]:
artists = list(map(lambda a: a[0], database.get_distinct_artists_id()))
albums = list(map(lambda a: a[0], database.get_distinct_albums_id()))
songs = list(map(lambda a: a[0], database.get_distinct_songs_id()))

In [16]:
for idx in range(len(artists)//50+1):
    batch = artists[idx*50:(idx+1)*50]
    res = sp.artists(batch)
    for artist in res["artists"]:
        try:
            id = artist["id"]
            genres = ",".join(artist["genres"])
            name = artist["name"]
            popularity = artist["popularity"]
            followers = artist["followers"]["total"]
    
            database.artists_insert(ArtistInfo(id, name, genres, popularity, followers))
        except DBException as exception_db:
            print(exception_db)

<div style="background: rgb(232, 232, 255); padding-left: 10px; border: 1px solid black; font-family: Monospace; border-radius: 15px; box-shadow: 2px 2px 0px 0px rgba(0,0,0,0.7);">

## Populating Albums info

Here I also saved the info about **every track in every distinct album**. It generated the majority of the rows! ~300k

In [40]:
for idx in range(len(albums)//20+1):
    batch = albums[idx*20:(idx+1)*20]
    res = sp.albums(batch)
    for album in res["albums"]:
        try:
            id = album["id"]
            name = album["name"]
            release_date = album["release_date"]
            total_tracks = album["tracks"]["total"]
            genres = ",".join(album["genres"])
            popularity = album["popularity"]
    
            database.albums_insert(AlbumInfo(id, name, release_date, total_tracks, genres, popularity))
        except DBException as exception_db:
            print(exception_db)

        for song in album["tracks"]["items"]:
            try:
                song_id = song["id"]
                album_id = album["id"]
                artist_id = song["artists"][0]["id"]
                title = song["name"]
                release_date = album["release_date"]
                featured = int(len(song["artists"]) > 1)
                popularity = -1

                database.songs_insert(IDSongInfo(song_id, album_id, artist_id, title, release_date, featured, popularity))
                
                
            except DBException as exception_db_song:
                continue

('58RXhgjtKkp72kiSl1Eery', 'Basi musicale nello stilo dei vari artisti (instrumental karaoke tracks) Vol. 114', 'UNIQUE constraint failed: albums.album_spotify_id')


<div style="background: rgb(232, 232, 255); padding-left: 10px; border: 1px solid black; font-family: Monospace; border-radius: 15px; box-shadow: 2px 2px 0px 0px rgba(0,0,0,0.7);">

### Unlucky, there was a little issue

By collecting the data about songs from albums, there was no given data about the popularity, So I though that it is not a real problem, knowing that there is a **get_track** method in Spotify API, where I can get the info about the track (with popularity), and also I can provide **50 songs** at once!

In [17]:
query = """
        SELECT song_spotify_id
        FROM songs
        WHERE popularity = -1;
        """

In [18]:
incomplete_songs = list(map(lambda s: s[0], database.get_query_database(query)))
start = datetime.now()
for idx in range(len(incomplete_songs)//50+1):
    sleep(0.5)
    if idx % 100 == 0 and idx != 0:
        print("No:", idx)
        sleep(5)
    try:
        batch = incomplete_songs[50*idx:50*(idx+1)]
        songs = sp.tracks(batch)
    except Exception as exception:
        print(exception)
        sleep(10)
        continue

    for song in songs["tracks"]:
        try:
            id = song["id"]
            popularity = song["popularity"]
            database.update_song_popularity(id, popularity)
        except DBException as exception_db:
            print(exception_db)
            continue
print(datetime.now()-start)

No: 100
No: 200
No: 300
No: 400
No: 500
No: 600
No: 700
No: 800
No: 900
No: 1000
No: 1100
No: 1200
No: 1300
No: 1400
No: 1500
No: 1600
No: 1700
No: 1800
0:31:26.027570


<div style="background: rgb(232, 232, 255); padding-left: 10px; border: 1px solid black; font-family: Monospace; border-radius: 15px; box-shadow: 2px 2px 0px 0px rgba(0,0,0,0.7);">

## Getting the song technical features
Even though, the maximum number of songs I can request is 100 at once, I got banned for some time for too many requests. Had to continue the next day.

In [5]:
query_2 = """SELECT song_spotify_id
             FROM songs
             WHERE song_spotify_id NOT IN (
                 SELECT song_spotify_id
                 FROM songs_features);"""

In [6]:
missing_features_songs = list(map(lambda s: s[0], database.get_query_database(query_2)))
for idx in range(len(missing_features_songs)//100+1):
    if idx % 50 == 0 and idx != 0:
        sleep(3)
        print(idx, datetime.now())
    try:
        batch = missing_features_songs[idx*100:(idx+1)*100]
        features = sp.audio_features(batch)
    except Exception as exception:
        print(exception)
        sleep(5)

    for f in features:
        try:
            database.songs_features_insert(SongFeatures(f))
        except DBException as exception_db:
            print(exception_db)
            continue
        except ValueError as exception_value:
            continue

Max Retries reached


http status: 429, code:-1 - /v1/audio-features/?ids=3VrsEoh3M6BcH5QLu4Zi4q,3W02G5JB1kQc13NVACA94Y,3ZYvax9W5YigQ9bwS9byeX,3ZnZjahZlRbcH4tqycNdfC,3eZk3GxGcwVDPa1xIK4Fnp,3gUFSe8zIIClWgPj7CNYFi,3hsY5MoqiOpIn4bcRtizMZ,3jWyLzK2XnBYg0M02wPjbn,3neKhXDhXf17BiKa1noeUX,3oIRRjzbZhl07MqcEG7nhH,3pmP3hLi7HYOZYCuxZm1aE,3rVXcHkfWFhHx9Str4J1Wr,3t0l1FEYQQpjmCJtsPIvN3,3t9JBaGfPOEQbNIexSOgDf,3tpof5OFTKWH4U0rPFTEYQ,3uh7YcFzAWHGg7spVzPfqP,3ujliGKxByL8Va0phmVSEv,3w1F13Rv4JO51g5ksX0JzL,3w39hZDGqukIsEajUX5FH9,3xWfLEFVduwUuVOYwl29FD,3yS2615iCahNh10KyfNQA5,3zRY4aZB5RfGlJXafJnfY0,439gEXy3QO4CwMA2Fd8yDS,43BebLBBuSCg4qauLd1eqc,43LajfnvG9QTQ5VLqNUdHs,44P02BdN2fK0IMwYDUVIT2,46MCRTlHg2RK4c9eOmrZaU,46taYp5BODIX9i43VFPGDV,4C9pMTWyEBsYJ6NAh0uIWE,4Gyevf6hfKBzh69K79twjP,4HsvrYvpcFefRvQSBkTXyJ,4MyiAZ0G0LhwfTfEJxrN8C,4PM5CltOCcwkFJBfAyAhhJ,4QTUKOwsEsZjLe2YV65Oww,4SFbevneLmLf9upVuzVK2t,4SJTFnmmfxotIdw2psMeDp,4TBawOJBjdMe9ZetXnNP0R,4TUPsBH8KWrV7mXreb4mlk,4TZbJI4hnIysvYCThJyzxI,4YQWW24PWJ1Z0BXudSj74U,4YqCQOLbUGSWFGUJntT4Za,4ZUng

KeyboardInterrupt: 

In [13]:
!git status

On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   dataset-creator.ipynb
	modified:   db_management/__pycache__/db.cpython-311.pyc
	modified:   db_management/db.py

no changes added to commit (use "git add" and/or "git commit -a")


In [14]:
!git add .

In [15]:
!git commit -m "features, lots of data"

[master 5d3400d] features, lots of data
 3 files changed, 1333 insertions(+), 324 deletions(-)


In [16]:
!git push origin master

To github.com:milonpabis/songs-across-years.git
   8fd8409..5d3400d  master -> master
